### 🔹 Importación de librerías  
En esta sección cargamos las librerías necesarias para el manejo de datos.  
`pandas` nos permite trabajar con archivos CSV y realizar operaciones de integración, limpieza y análisis.

In [1]:
import pandas as pd

# Cargar el dataset combinado
data = pd.read_csv("Combined_Dataset.csv")

print("\n=== 1. Vista general del dataset ===")
print(data.info())


=== 1. Vista general del dataset ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10706 entries, 0 to 10705
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sender    10375 non-null  object
 1   receiver  9076 non-null   object
 2   date      10223 non-null  object
 3   subject   10647 non-null  object
 4   body      10705 non-null  object
 5   urls      10706 non-null  int64 
 6   label     10706 non-null  int64 
 7   origen    10706 non-null  object
dtypes: int64(2), object(6)
memory usage: 669.3+ KB
None


## 2. Vista preliminar del contenido

Revisamos las primeras 5 filas para observar la estructura básica del dataset, ejemplos de valores reales y posibles inconsistencias.  
Esto ayuda a validar que la concatenación de los datasets se realizó correctamente.

In [2]:
print("\n=== 2. Primeras 5 filas ===")
print(data.head())


=== 2. Primeras 5 filas ===
                                              sender  \
0  Mail System Internal Data <MAILER-DAEMON@monke...   
1                        cPanel <service@cpanel.com>   
2    Microsoft Outlook <recepcao@unimedceara.com.br>   
3                     Ann Garcia <AnGarcia@mcoe.org>   
4                 "USAA" <usaaacctupdate@sccu4u.com>   

                                 receiver  \
0                                     NaN   
1                         jose@monkey.org   
2                                     NaN   
3     "info@maaaaa.org" <info@maaaaa.org>   
4  Recipients <usaaacctupdate@sccu4u.com>   

                                    date  \
0             28 Sep 2017 09:57:25 -0400   
1        Fri, 30 Oct 2015 00:00:48 -0500   
2  Fri, 30 Oct 2015 06:21:59 -0300 (BRT)   
3        Fri, 30 Oct 2015 14:54:33 +0000   
4        Fri, 30 Oct 2015 14:02:33 -0500   

                                             subject  \
0  DON'T DELETE THIS MESSAGE -- FOLDER INT

## 3. Distribución de clases (ham vs phishing)

Para evaluar el balance del dataset, contamos cuántas instancias pertenecen a cada clase (`label`):
- `0` = correo legítimo (ham)
- `1` = phishing / fraudulento

Esto es importante para determinar si se requieren técnicas de balanceo más adelante.

In [3]:
print("\n=== 3. Distribución de clases ===")
print(data['label'].value_counts())


=== 3. Distribución de clases ===
label
1    6615
0    4091
Name: count, dtype: int64


## 4. Análisis de valores nulos

Revisamos la cantidad total de valores faltantes por columna y su porcentaje relativo.  
Identificar estos valores es fundamental para la etapa de limpieza y feature engineering.

In [4]:
print("\n=== 4. Valores nulos por columna ===")
print(data.isna().sum())

print("\n=== 5. Porcentaje de valores nulos ===")
print((data.isna().mean()*100).round(2))


=== 4. Valores nulos por columna ===
sender       331
receiver    1630
date         483
subject       59
body           1
urls           0
label          0
origen         0
dtype: int64

=== 5. Porcentaje de valores nulos ===
sender       3.09
receiver    15.23
date         4.51
subject      0.55
body         0.01
urls         0.00
label        0.00
origen       0.00
dtype: float64


## 5. Detección de duplicados

Buscamos duplicados tanto:
- por el cuerpo del mensaje (`body`)  
- como duplicados completos de filas

Esto permite depurar el dataset y evitar sesgos durante el modelado.

In [5]:
print("\n=== 6. Duplicados ===")
print("Duplicados por cuerpo:", data.duplicated(subset=['body']).sum())
print("Duplicados completos:", data.duplicated().sum())


=== 6. Duplicados ===
Duplicados por cuerpo: 2
Duplicados completos: 0


## 6. Características de longitud de texto

Creamos dos nuevas variables:
- `subject_len`: longitud del asunto  
- `body_len`: longitud del cuerpo del mensaje

Estas características son útiles para detectar patrones típicos de correos maliciosos (por ejemplo, cuerpos muy extensos o muy cortos).

In [6]:
print("\n=== 7. Longitud de subject y body ===")
data['subject_len'] = data['subject'].astype(str).apply(len)
data['body_len'] = data['body'].astype(str).apply(len)
print(data[['subject_len', 'body_len']].describe())


=== 7. Longitud de subject y body ===
        subject_len      body_len
count  10706.000000  1.070600e+04
mean      33.189520  2.820186e+03
std       31.926531  4.591314e+04
min        2.000000  1.000000e+00
25%       19.000000  7.230000e+02
50%       30.000000  1.429500e+03
75%       44.000000  2.687750e+03
max     2657.000000  4.599644e+06


## 7. Conteo aproximado de URLs por correo

Calculamos la cantidad de URLs presentes en cada correo a partir de la columna `urls`.  
Esto ayuda a identificar campañas de phishing que contienen múltiples enlaces sospechosos.

In [7]:
print("\n=== 8. Conteo de URLs ===")
data['num_urls'] = data['urls'].astype(str).apply(lambda x: len(str(x).split()) if x != 'nan' else 0)
print(data['num_urls'].describe())


=== 8. Conteo de URLs ===
count    10706.0
mean         1.0
std          0.0
min          1.0
25%          1.0
50%          1.0
75%          1.0
max          1.0
Name: num_urls, dtype: float64


## 8. Guardado de dataset enriquecido

Guardamos el dataset actualizado con las nuevas características (`subject_len`, `body_len`, `num_urls`) en un nuevo archivo llamado `Combined_Dataset_Features.csv`.  
Este archivo será utilizado en la siguiente etapa de limpieza avanzada y feature engineering.

In [8]:
# Guardamos con nuevas columnas
data.to_csv("Combined_Dataset_Features.csv", index=False)
print("\n>>> Archivo guardado: Combined_Dataset_Features.csv")


>>> Archivo guardado: Combined_Dataset_Features.csv
